In [3]:
!pip install -r ./requirements.txt -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.3/328.3 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 975.5/975.5 kB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.9/45.9 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 202.7/202.7 kB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.4/216.4 kB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 36.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 20.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 337.4/337.4 kB 23.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.5/127.5 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 32.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.

In [4]:
import os
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv(), override=True)
pinecone_api_key = os.environ.get('PINECONE_API_KEY')

In [5]:
pip show pinecone-client

Name: pinecone-client
Version: 4.1.2
Summary: Pinecone client and SDK
Home-page: https://www.pinecone.io
Author: Pinecone Systems, Inc.
Author-email: support@pinecone.io
License: Apache-2.0
Location: /usr/local/lib/python3.10/dist-packages
Requires: certifi, pinecone-plugin-interface, tqdm, typing-extensions, urllib3
Required-by: 


In [6]:
from pinecone import Pinecone
pc = Pinecone(pinecone_api_key)

Using Pinecone Indexes: An index is the highest-level organizational unit of vector data in Pinecone. It accepts and stores vectors, serves queries over the vectors it contains, and does other vector operations over its contents. It help to significantly increase the speed of the search process of similarity search with only a minimal tradeoff in search accuracy (HNSW index), or efficiently store many subsets of data in a small memory footprint (flat index)

Vector embeddings are produced by embedding models to summarize objects such as text, image, video, etc.

In [9]:
pc.list_indexes()

{'indexes': []}

Create Index (Note: serverless indexes scale automatically based on usage, and you pay only for the amount of data stored and operations performed, with no minimums.)

In [16]:
from pinecone import ServerlessSpec
index_name = 'langchain'
if index_name not in pc.list_indexes().names():
  print(f'Creating index {index_name}...')
  pc.create_index(name=index_name, dimension=1536, metric='cosine', spec=ServerlessSpec(cloud='aws', region='us-east-1'))
  print('Index created.')
else:
  print(f'Index {index_name} already exists.')

Creating index langchain...
Index created.


Delete Index

In [14]:
index_name = 'langchain'
if index_name in pc.list_indexes().names():
  print(f'Deleting index {index_name}...')
  pc.delete_index(index_name)
  print('Index deleted.')
else:
  print(f'Index {index_name} does not exist.')

Deleting index langchain...
Index deleted.


Selecting Index

In [17]:
index = pc.Index(index_name)
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {},
 'total_vector_count': 0}

Working with vectors

Inserting vectors

In [20]:
import random
vectors = [[random.random() for _ in range(1536)] for v in range(5)]
# print(vectors)
ids = list('abcde')
index_name = 'langchain'
index = pc.Index(index_name)
index.upsert(vectors=zip(ids, vectors))

{'upserted_count': 5}

Updating vectors

In [21]:
index.upsert(vectors=[('c', [0.5]*1536)])

{'upserted_count': 1}

Fetching vectors

In [23]:
#index.fetch(ids=['a', 'c'])

Deleting vectors

In [24]:
index.delete(ids=['b'])

{}

In [25]:
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 4}},
 'total_vector_count': 4}

In [26]:
index.fetch(ids=['y'])

{'namespace': '', 'usage': {'read_units': 1}, 'vectors': {}}

Query

In [31]:
query_vector = [random.random() for _ in range(1536)]
index.query(vector=query_vector, top_k=3, include_values=False)

{'matches': [{'id': 'c', 'score': 0.864851117, 'values': []},
             {'id': 'e', 'score': 0.762854934, 'values': []},
             {'id': 'd', 'score': 0.742488325, 'values': []}],
 'namespace': '',
 'usage': {'read_units': 5}}

Namespaces

In [7]:
import random
index = pc.Index('langchain')

vectors = [[random.random() for _ in range(1536)] for v in range(5)]
ids = list('abcde')
index.upsert(vectors=zip(ids, vectors))

{'upserted_count': 5}

In [17]:
vectors = [[random.random() for _ in range(1536)] for v in range(3)]
ids = list('xyz')
index.upsert(vectors=zip(ids, vectors), namespace='first-namespace')

{'upserted_count': 3}

In [24]:
vectors = [[random.random() for _ in range(1536)] for v in range(2)]
ids = list('qp')
index.upsert(vectors=zip(ids, vectors), namespace='second-namespace')

{'upserted_count': 2}

In [25]:
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 5},
                'first-namespace': {'vector_count': 3},
                'second-namespace': {'vector_count': 2}},
 'total_vector_count': 10}

In [27]:
index.fetch(ids=['x'])

{'namespace': '', 'usage': {'read_units': 1}, 'vectors': {}}

In [29]:
# index.fetch(ids=['x'], namespace='first-namespace')

In [30]:
index.delete(delete_all=True, namespace='first-namespace')

{}

In [31]:
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 5},
                'second-namespace': {'vector_count': 2}},
 'total_vector_count': 7}